# Pascal Parts

In [2]:
import sys
sys.path.append('/home/akirchme/packages/pascal-part-py')
import os
import numpy as np
import matplotlib.pylab as plt
from skimage.io import imread
from tqdm import tqdm

from VOClabelcolormap import color_map
from anno import ImageAnnotation

from os import listdir
from os.path import isfile, join
import cv2

pascal_dir = '/home/akirchme/data/VOC2010'
pascal_annot_dir = f'{pascal_dir}/Annotations_Part'
pascal_img_dir = f'{pascal_dir}/JPEGImages'
annotations = [f for f in listdir(pascal_annot_dir) if isfile(join(f'{pascal_annot_dir}', f))]

output_dir = '/grogu/user/akirchme/art3d_pascal'
classes = ['horse', 'car', 'dog', 'bird']
min_size = 128

for cls in classes:
    os.makedirs(f'{output_dir}/images/{cls}', exist_ok=True)
    os.makedirs(f'{output_dir}/fg/{cls}', exist_ok=True)
    os.makedirs(f'{output_dir}/masks/{cls}', exist_ok=True)
    
#class_imgs = {cls:[] for cls in classes}
class_imgs = {cls:0 for cls in classes}

for img_id in tqdm(annotations):
    img_fn = img_id.split('.')[0]
    annot = ImageAnnotation(f'{pascal_img_dir}/{img_fn}.jpg', f'{pascal_annot_dir}/{img_fn}.mat')
    for obj in annot.objects:
        cls = obj.class_name
        if cls in classes:
            mask = obj.mask == 1
            if mask.sum() >= min_size*min_size:
                image = np.copy(annot.im)
                i = class_imgs[cls]
                cv2.imwrite(f'{output_dir}/images/{cls}/{i}.png', cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
                image[mask == 0] = 0
                cv2.imwrite(f'{output_dir}/fg/{cls}/{i}.png', cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
                np.save(f'{output_dir}/masks/{cls}/{i}.npy', mask)
                class_imgs[cls] += 1
                #class_imgs[cls].append(img)

100%|█████████████████████████████████████████████████████████████████████████████| 10103/10103 [04:10<00:00, 40.28it/s]
